# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests



# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")


# Display sample data
city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,plettenberg bay,-34.0527,23.3716,14.44,88,100,2.16,ZA,1680981262
1,1,yulara,-25.2406,130.9889,18.89,43,2,4.72,AU,1680981262
2,2,rikitea,-23.1203,-134.9692,24.64,63,17,7.22,PF,1680981262
3,3,hermanus,-34.4187,19.2345,13.78,67,100,1.58,ZA,1680981262
4,4,sehithwa,-20.4667,22.7167,27.50,31,90,1.83,BW,1680981263


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [73]:
# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng","Lat", geo=True, size="humidity")



#Display the map
map_plot





ValueError: cannot convert float NaN to integer

:Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[city_data_df["Max Temp"] < 27]      
city_data_df = city_data_df.loc[city_data_df["Max Temp"] > 21]
city_data_df = city_data_df.loc[city_data_df["Wind Speed"] < 5]
city_data_df = city_data_df.loc[city_data_df["Cloudiness"] == 0]



# # Drop any rows with null values
city_data_df = city_data_df.dropna()


# Display sample data
city_data_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API   
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "radius": radius,
    "types": "lodging",
    "key": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point_distance:{lng},{lat}"
    params["bias"] = f"proximity:{lng},{lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    name_address = name_address["features"][0]["properties"]["name"]
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
# %%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points("Lng","Lat", geo=True, size="Humidity", hover_cols=["City", "Country", "Hotel Name"])

# Display the map
map_plot

ValueError: cannot convert float NaN to integer

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)